In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv')
campaign_data = pd.read_csv('campaign_data.csv')
coupon_item_mapping = pd.read_csv('coupon_item_mapping.csv')
customer_demographics = pd.read_csv('customer_demographics.csv')
customer_transaction_data = pd.read_csv('customer_transaction_data.csv')
item_data = pd.read_csv('item_data.csv')
test_data = pd.read_csv('test.csv')

In [3]:
ti = pd.merge(customer_transaction_data, item_data, on='item_id', how='left')

In [4]:
ti['category'] = ti['category'].replace({'Packaged Meat': 'Gift Cards', 'Bakery': 'Digital Gold', 'Dairy, Juices & Snacks': 'Electricity Bill', 'Natural Products': 'Jewelry', 'Meat': 'Insurance'})
desired_categories = ['Gift Cards', 'Insurance', 'Digital Gold', 'Electricity Bill', 'Jewelry', 'Pharmaceutical', 'Grocery']
ti = ti[ti['category'].isin(desired_categories)]
ti['category'] = ti['category'].replace('Pharmaceutical', 'Pharmacy')

In [7]:
ti['category'].value_counts()

Grocery             942176
Pharmacy            155245
Gift Cards           66110
Jewelry              52981
Insurance            23056
Electricity Bill     20465
Digital Gold         18218
Name: category, dtype: int64

In [8]:
category_avg = ti.groupby('category').agg({'selling_price': 'mean', 'other_discount': 'mean', 'coupon_discount': 'mean'})
category_avg

,selling_price,other_discount,coupon_discount
category,,,
Digital Gold,117.956141,-9.962247,-0.102159
Electricity Bill,107.100653,-12.712502,-0.237140
Gift Cards,134.488465,-35.495811,-0.350940
Grocery,91.521779,-17.476031,-0.599109
Insurance,202.534326,-39.835106,-0.066664
Jewelry,86.902200,-7.616079,-0.148649
Pharmacy,147.324046,-11.652425,-1.090751


In [9]:
np.random.seed(42)
grocery_rows = ti[ti['category'] == 'Grocery']
num_rows_to_delete = int(0.8 * len(grocery_rows))
rows_to_delete = np.random.choice(grocery_rows.index, size=num_rows_to_delete, replace=False)
ti = ti.drop(rows_to_delete)

In [10]:
jewelry_rows = ti['category'] == 'Jewelry'
ti.loc[jewelry_rows, 'selling_price'] *= 80
ti.loc[jewelry_rows, 'other_discount'] *= 80
ti.loc[jewelry_rows, 'coupon_discount'] *= 80

In [11]:
g_rows = ti['category'] == 'Grocery'
ti.loc[g_rows, 'selling_price'] *= 3
ti.loc[g_rows, 'other_discount'] *= 3
ti.loc[g_rows, 'coupon_discount'] *= 3

In [12]:
gi_rows = ti['category'] == 'Gift Cards'
ti.loc[gi_rows, 'selling_price'] *= 8
ti.loc[gi_rows, 'other_discount'] *= 8
ti.loc[gi_rows, 'coupon_discount'] *= 8

In [13]:
ei_rows = ti['category'] == 'Electricity Bill'
ti.loc[ei_rows, 'selling_price'] *= 12
ti.loc[ei_rows, 'other_discount'] *= 12
ti.loc[ei_rows, 'coupon_discount'] *= 12

In [14]:
i_rows = ti['category'] == 'Insurance'
ti.loc[i_rows, 'selling_price'] *= 25
ti.loc[i_rows, 'other_discount'] *= 25
ti.loc[i_rows, 'coupon_discount'] *= 25

In [15]:
dg_rows = ti['category'] == 'Digital Gold'
ti.loc[dg_rows, 'selling_price'] *= 80
ti.loc[dg_rows, 'other_discount'] *= 80
ti.loc[dg_rows, 'coupon_discount'] *= 80

In [16]:
ti['category'].value_counts()

Grocery             188436
Pharmacy            155245
Gift Cards           66110
Jewelry              52981
Insurance            23056
Electricity Bill     20465
Digital Gold         18218
Name: category, dtype: int64

In [17]:
category_avg = ti.groupby('category').agg({'selling_price': 'mean', 'other_discount': 'mean', 'coupon_discount': 'mean'})
category_avg

,selling_price,other_discount,coupon_discount
category,,,
Digital Gold,9436.491250,-796.979778,-8.172708
Electricity Bill,1285.207840,-152.550028,-2.845674
Gift Cards,1075.907721,-283.966485,-2.807522
Grocery,274.329262,-52.534853,-1.803596
Insurance,5063.358139,-995.877646,-1.666594
Jewelry,6952.176033,-609.286318,-11.891886
Pharmacy,147.324046,-11.652425,-1.090751


In [20]:
ti = ti.drop(['brand', 'brand_type'], axis=1)

In [22]:
ti = ti.sort_values(by=['customer_id'])
ti

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category
193877,2012-05-17,1,28590,1,11369.60,0.00,0.00,Digital Gold
417865,2012-08-12,1,52350,1,307.77,-204.09,0.00,Grocery
848584,2013-01-10,1,5525,1,1136.96,0.00,0.00,Gift Cards
848585,2013-01-10,1,6089,1,74.45,-19.23,-26.71,Pharmacy
848587,2013-01-10,1,7492,1,63.76,0.00,0.00,Pharmacy
...,...,...,...,...,...,...,...,...
894526,2013-01-26,1582,25356,1,852.00,0.00,0.00,Gift Cards
649060,2012-11-02,1582,7512,1,76.58,0.00,0.00,Pharmacy
649061,2012-11-02,1582,15958,1,89.05,-31.70,0.00,Pharmacy
487505,2012-09-04,1582,10944,1,41.67,0.00,0.00,Grocery


In [29]:
ti['customer_id'].nunique()

1582

In [25]:
train_campaign = pd.merge(train_data, campaign_data, on='campaign_id', how='left')

In [31]:
train_campaign = train_campaign.sort_values(by=['customer_id', 'start_date'])
train_campaign

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date
51584,84658,29,416,1,0,Y,08/10/12,30/11/12
67721,111193,29,458,1,0,Y,08/10/12,30/11/12
58566,96238,29,868,1,0,Y,08/10/12,30/11/12
61514,101106,29,643,1,0,Y,08/10/12,30/11/12
64676,106173,29,982,1,0,Y,08/10/12,30/11/12
...,...,...,...,...,...,...,...,...
51364,84284,13,890,1582,0,X,19/05/13,05/07/13
21030,34529,13,160,1582,0,X,19/05/13,05/07/13
17547,28849,13,151,1582,0,X,19/05/13,05/07/13
73895,121355,13,114,1582,0,X,19/05/13,05/07/13


In [33]:
train_campaign_item = pd.merge(train_campaign, coupon_item_mapping, on='coupon_id', how='left')

In [36]:
train_campaign_item

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,item_id
0,84658,29,416,1,0,Y,08/10/12,30/11/12,933
1,84658,29,416,1,0,Y,08/10/12,30/11/12,942
2,84658,29,416,1,0,Y,08/10/12,30/11/12,6443
3,84658,29,416,1,0,Y,08/10/12,30/11/12,10545
4,84658,29,416,1,0,Y,08/10/12,30/11/12,14500
...,...,...,...,...,...,...,...,...,...
6420689,121355,13,114,1582,0,X,19/05/13,05/07/13,7721
6420690,121355,13,114,1582,0,X,19/05/13,05/07/13,40025
6420691,121355,13,114,1582,0,X,19/05/13,05/07/13,17985
6420692,91060,13,561,1582,0,X,19/05/13,05/07/13,57307


In [37]:
ti

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category
193877,2012-05-17,1,28590,1,11369.60,0.00,0.00,Digital Gold
417865,2012-08-12,1,52350,1,307.77,-204.09,0.00,Grocery
848584,2013-01-10,1,5525,1,1136.96,0.00,0.00,Gift Cards
848585,2013-01-10,1,6089,1,74.45,-19.23,-26.71,Pharmacy
848587,2013-01-10,1,7492,1,63.76,0.00,0.00,Pharmacy
...,...,...,...,...,...,...,...,...
894526,2013-01-26,1582,25356,1,852.00,0.00,0.00,Gift Cards
649060,2012-11-02,1582,7512,1,76.58,0.00,0.00,Pharmacy
649061,2012-11-02,1582,15958,1,89.05,-31.70,0.00,Pharmacy
487505,2012-09-04,1582,10944,1,41.67,0.00,0.00,Grocery


In [51]:
merge = pd.merge(ti, train_campaign_item, on=['customer_id', 'item_id'], how='inner')

In [52]:
merge

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category,id,campaign_id,coupon_id,redemption_status,campaign_type,start_date,end_date
0,2013-01-10,1,5525,1,1136.96,0.00,0.00,Gift Cards,102278,13,24,0,X,19/05/13,05/07/13
1,2012-04-04,1,58644,2,534.30,-211.59,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13
2,2012-05-26,1,58644,1,319.50,-53.43,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13
3,2012-04-21,1,58644,1,267.15,-105.78,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13
4,2012-06-14,1,58644,1,319.50,-53.43,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62647,2013-01-26,1582,11712,2,29.92,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13
62648,2012-11-02,1582,32830,1,640.80,-35.62,-35.62,Pharmacy,82908,10,662,0,Y,08/04/13,10/05/13
62649,2013-01-26,1582,59550,2,29.92,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13
62650,2013-01-26,1582,25559,4,59.84,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13


In [40]:
merge = merge.sort_values(by=['customer_id', 'start_date'])
merge

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category,id,campaign_id,coupon_id,redemption_status,campaign_type,start_date,end_date
55,2012-10-18,1,56179,1,362.25,-32.07,0.00,Grocery,96238,29,868,0,Y,08/10/12,30/11/12
56,2012-11-20,1,56179,1,362.25,-32.07,0.00,Grocery,96238,29,868,0,Y,08/10/12,30/11/12
57,2013-02-04,1,56179,1,362.25,-32.07,0.00,Grocery,96238,29,868,0,Y,08/10/12,30/11/12
58,2012-04-18,1,56179,1,308.82,-85.50,0.00,Grocery,96238,29,868,0,Y,08/10/12,30/11/12
59,2013-02-16,1,56179,1,362.25,-32.07,0.00,Grocery,96238,29,868,0,Y,08/10/12,30/11/12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62628,2013-05-22,1582,22340,2,382.56,-64.11,0.00,Grocery,47762,9,578,0,Y,11/03/13,12/04/13
62641,2013-04-23,1582,28623,4,1282.32,-718.11,0.00,Grocery,24249,9,705,0,Y,11/03/13,12/04/13
62651,2013-04-09,1582,10491,1,135.00,0.00,0.00,Pharmacy,44511,9,691,0,Y,11/03/13,12/04/13
62630,2013-03-25,1582,56886,1,267.15,-95.10,-106.86,Grocery,1641,8,674,1,X,16/02/13,05/04/13


In [41]:
cashback_p = pd.read_excel('target_data_custom_modified.xlsx')

In [43]:
tab = cashback_p
tab['category'] = tab['category'].replace({'Packaged Meat': 'Gift Cards', 'Bakery': 'Digital Gold', 'Dairy, Juices & Snacks': 'Electricity Bill', 'Natural Products': 'Jewelry', 'Meat': 'Insurance'})
desired_categories = ['Gift Cards', 'Insurance', 'Digital Gold', 'Electricity Bill', 'Jewelry', 'Pharmaceutical', 'Grocery']
tab = tab[tab['category'].isin(desired_categories)]

In [44]:
tab['category'] = tab['category'].replace('Pharmaceutical', 'Pharmacy')

C:\Users\Home\AppData\Local\Temp\ipykernel_10284\2577117585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tab['category'] = tab['category'].replace('Pharmaceutical', 'Pharmacy')


In [46]:
tab = tab.sort_values(by=['customer_id', 'start_date'])
tab

,id,campaign_id,coupon_id,customer_id,start_date,end_date,category,redemption_status,cashback percentage,null reassignment
3178,4674,29,597,1,2012-10-08,2012-11-30,Grocery,0,0.491759,0
4583,6741,29,896,1,2012-10-08,2012-11-30,Electricity Bill,0,0.302298,1
7055,10185,29,696,1,2012-10-08,2012-11-30,Electricity Bill,0,0.302298,1
11335,16239,29,750,1,2012-10-08,2012-11-30,Grocery,0,0.302298,1
13256,19022,29,443,1,2012-10-08,2012-11-30,Grocery,0,0.166667,0
...,...,...,...,...,...,...,...,...,...,...
63041,91060,13,561,1582,2013-05-19,2013-07-05,Pharmacy,0,0.302298,1
71907,103994,13,766,1582,2013-05-19,2013-07-05,Grocery,0,0.283554,0
83829,121355,13,114,1582,2013-05-19,2013-07-05,Grocery,0,0.302298,1
84366,122144,13,511,1582,2013-05-19,2013-07-05,Grocery,0,0.314590,0


In [56]:
merge['cashback percentage']=-1*merge['coupon_discount']/(merge['selling_price'])

In [58]:
merge['cashback percentage'].nunique()

608

In [61]:
merge['redemption_status'].value_counts()

0    56101
1     6551
Name: redemption_status, dtype: int64

In [64]:
merge

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,category,id,campaign_id,coupon_id,redemption_status,campaign_type,start_date,end_date,cashback percentage
0,2013-01-10,1,5525,1,1136.96,0.00,0.00,Gift Cards,102278,13,24,0,X,19/05/13,05/07/13,-0.000000
1,2012-04-04,1,58644,2,534.30,-211.59,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13,-0.000000
2,2012-05-26,1,58644,1,319.50,-53.43,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13,-0.000000
3,2012-04-21,1,58644,1,267.15,-105.78,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13,-0.000000
4,2012-06-14,1,58644,1,319.50,-53.43,0.00,Grocery,94621,8,772,1,X,16/02/13,05/04/13,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62647,2013-01-26,1582,11712,2,29.92,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13,-0.000000
62648,2012-11-02,1582,32830,1,640.80,-35.62,-35.62,Pharmacy,82908,10,662,0,Y,08/04/13,10/05/13,0.055587
62649,2013-01-26,1582,59550,2,29.92,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13,-0.000000
62650,2013-01-26,1582,25559,4,59.84,0.00,0.00,Pharmacy,16149,10,934,0,Y,08/04/13,10/05/13,-0.000000


In [65]:
merge_del = merge.drop(['date', 'item_id', 'quantity', 'id', 'campaign_id', 'coupon_id', 'campaign_type', 'cashback percentage'], axis=1)

In [71]:
merge_del = merge_del.sort_values(by=['customer_id', 'start_date'])

In [67]:
xg = pd.read_excel('XGBoost.xlsx')

In [69]:
xg = xg.drop(['Unnamed: 0'], axis=1)

In [70]:
xg

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.302298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61254,1580,2013-02-16,213.72,Pharmacy,Pharmacy,11,4,0,0.00000,Jewelry,...,661.10,5670.4,0.0,1136.96,0.0,753.36,1,30,Jewelry,0.478304
61255,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0,25,Pharmacy,0.302298
61256,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0,25,Grocery,0.339219
61257,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0,25,Grocery,0.301381


In [87]:
xg['redemption_status'] = merge_del['redemption_status'].head(61259)

In [88]:
xg['redemption_status'].value_counts()

0.0    55094
1.0     6143
Name: redemption_status, dtype: int64

In [80]:
xg

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.302298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61254,1580,2013-02-16,213.72,Pharmacy,Pharmacy,11,4,0,0.00000,Jewelry,...,661.10,5670.4,0.0,1136.96,0.0,753.36,0.0,30,Jewelry,0.478304
61255,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Pharmacy,0.302298
61256,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Grocery,0.339219
61257,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Grocery,0.301381


In [81]:
# Define the conditions for Digital Gold and Jewelry categories
condition_dg_high_cb = (xg['category'] == 'Digital Gold') & (xg['cashback percentage'] > 0.5)
condition_dg_low_cb = (xg['category'] == 'Digital Gold') & (xg['cashback percentage'] < 0.5) & (xg['cashback percentage'] > 0.25)
condition_jewelry_high_cb = (xg['category'] == 'Jewelry') & (xg['cashback percentage'] > 0.5)
condition_jewelry_low_cb = (xg['category'] == 'Jewelry') & (xg['cashback percentage'] < 0.5) & (xg['cashback percentage'] > 0.25)

# Modify the redemption_status based on the conditions
xg.loc[condition_dg_high_cb, 'redemption_status'] = xg.loc[condition_dg_high_cb, 'redemption_status'].sample(frac=0.4)
xg.loc[condition_dg_low_cb, 'redemption_status'] = xg.loc[condition_dg_low_cb, 'redemption_status'].sample(frac=0.2)
xg.loc[condition_jewelry_high_cb, 'redemption_status'] = xg.loc[condition_jewelry_high_cb, 'redemption_status'].sample(frac=0.4)
xg.loc[condition_jewelry_low_cb, 'redemption_status'] = xg.loc[condition_jewelry_low_cb, 'redemption_status'].sample(frac=0.2)


In [82]:
xg['redemption_status'].value_counts()

0.0    50899
1.0     5668
Name: redemption_status, dtype: int64

In [83]:
xg

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.302298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61254,1580,2013-02-16,213.72,Pharmacy,Pharmacy,11,4,0,0.00000,Jewelry,...,661.10,5670.4,0.0,1136.96,0.0,753.36,NaN,30,Jewelry,0.478304
61255,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Pharmacy,0.302298
61256,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Grocery,0.339219
61257,1580,2013-05-19,375.09,Gift Cards,Grocery,3,2,0,0.00000,Pharmacy,...,81.57,0.0,0.0,0.00,0.0,667.86,0.0,25,Grocery,0.301381


In [84]:
# Define the conditions for Gift Cards category
condition_gc_high_cb = (xg['category'] == 'Gift Cards') & (xg['cashback percentage'] > 0.6)
condition_gc_low_cb = (xg['category'] == 'Gift Cards') & (xg['cashback percentage'] < 0.6) & (xg['cashback percentage'] > 0.25)

# Modify the redemption_status based on the conditions
xg.loc[condition_gc_high_cb, 'redemption_status'] = xg.loc[condition_gc_high_cb].sample(frac=0.6)['redemption_status']
xg.loc[condition_gc_low_cb, 'redemption_status'] = xg.loc[condition_gc_low_cb].sample(frac=0.3)['redemption_status']


In [91]:
xg['redemption_status'].value_counts()

0.0    59094
1.0     2143
Name: redemption_status, dtype: int64

In [90]:
# Define the condition for redemption_status = 1.0
condition_rs_1 = (xg['redemption_status'] == 1.0)

# Randomly select 4000 rows that satisfy the condition and modify the redemption_status
random_indices = xg.loc[condition_rs_1].sample(n=4000).index
xg.loc[random_indices, 'redemption_status'] = 0.0


In [93]:
xg.head(25)

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.302298
5,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.242263
6,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.218340
7,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.346027
8,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.250000
9,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.222222


In [94]:
# Define the conditions for cashback percentage ranges
condition_cb_high = xg['cashback percentage'] > 0.85
condition_cb_low = (xg['cashback percentage'] < 0.85) & (xg['cashback percentage'] > 0.6)

# Randomly select rows and modify the redemption_status based on the conditions
high_indices = xg.loc[condition_cb_high].sample(frac=0.55).index
low_indices = xg.loc[condition_cb_low].sample(frac=0.35).index

xg.loc[high_indices, 'redemption_status'] = 1.0
xg.loc[low_indices, 'redemption_status'] = 1.0

# Define the condition for cashback percentage range
condition_cb = (xg['cashback percentage'] < 0.6) & (xg['cashback percentage'] > 0.4)

# Randomly select rows and modify the redemption_status based on the condition
indices = xg.loc[condition_cb].sample(frac=0.15).index
xg.loc[indices, 'redemption_status'] = 1.0


In [102]:
xg['redemption_status'].value_counts()

0.0    58647
1.0     2612
Name: redemption_status, dtype: int64

In [98]:
xg['redemption_status'] = xg['redemption_status'].fillna(0.0)


In [103]:
xg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61259 entries, 0 to 61258
Data columns (total 50 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   customer_id                                    61259 non-null  int64         
 1   start_date                                     61259 non-null  datetime64[ns]
 2   cashback_value_1m                              61259 non-null  float64       
 3   last_reward_transaction_category_1m            61259 non-null  object        
 4   reward_transaction_category_frequent_1m        61259 non-null  object        
 5   coupons_issued_1m                              61259 non-null  int64         
 6   coupons_collected_1m                           61259 non-null  int64         
 7   coupons_redeemed_1m                            61259 non-null  int64         
 8   net_redemption_rate_1m                         61259 non

In [101]:
condition_rs_1 = (xg['redemption_status'] == 1.0)

# Randomly select 4000 rows that satisfy the condition and modify the redemption_status
random_indices = xg.loc[condition_rs_1].sample(n=1000).index
xg.loc[random_indices, 'redemption_status'] = 0.0

In [104]:
xg.head(23)

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.302298
5,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.242263
6,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1.0,18,Grocery,0.218340
7,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.346027
8,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.250000
9,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0.0,18,Grocery,0.222222


In [114]:
xg.to_excel('coupon_redemption_data_for_xgboost.xlsx')

In [109]:
xgb = xg.drop_duplicates()


In [111]:
xgb['redemption_status'].value_counts()

0.0    38279
1.0     2400
Name: redemption_status, dtype: int64

In [112]:
xgb.to_excel('coupon_redemption_data_xgboost.xlsx')

In [2]:
df = pd.read_excel('coupon_redemption_data_for_xgboost.xlsx')

In [4]:
df = df.drop(['Unnamed: 0'], axis=1)

In [5]:
df.head()

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.196654
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.467270
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1,18,Grocery,0.211676
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.286542
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.302298


In [ ]:
import numpy as 

In [6]:
import numpy as np

# Set the random seed for reproducibility
np.random.seed(1)

# Generate random values for cashback_percentage
df['cashback_percentage'] = np.where(df['redemption_status'] == 0, np.random.uniform(0, 0.3, len(df)), np.random.uniform(0.15, 0.5, len(df)))

# Round the values to 5 decimal places
df['cashback_percentage'] = df['cashback_percentage'].round(5)


In [7]:
df1 = df

In [8]:
df1.head()

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback percentage,cashback_percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.196654,0.12511
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.467270,0.21610
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,5699.2,0.0,7029.92,4274.4,3117.06,1,18,Grocery,0.211676,0.25254
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.286542,0.09070
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.302298,0.04403


In [13]:
df1 = df1.drop(['cashback percentage'], axis=1)

In [14]:
df1.head()

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback_percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.12511
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.21610
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1,18,Grocery,0.25254
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.09070
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.04403


In [15]:
df1['redemption_status'].value_counts()

0    58743
1     2516
Name: redemption_status, dtype: int64

In [16]:
df1.to_excel('cashback_data_xgboost.xlsx')

In [18]:
df1['cashback_percentage'].max()

0.49991

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('cashback_data_xgboost.xlsx')

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
df['cashback_percentage'].max()

0.49991

In [6]:
df['redemption_status'].value_counts()

0    58743
1     2516
Name: redemption_status, dtype: int64

In [7]:
df.head()

,customer_id,start_date,cashback_value_1m,last_reward_transaction_category_1m,reward_transaction_category_frequent_1m,coupons_issued_1m,coupons_collected_1m,coupons_redeemed_1m,net_redemption_rate_1m,first_use_case_1m,...,Pharmacy_value_1m,Jewelry_value_1m,Insurance_value_1m,Gift cards_value_1m,Digital gold purchase_value_1m,Grocery_value_1m,redemption_status,days_difference,category,cashback_percentage
0,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.12511
1,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.21610
2,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,1,18,Grocery,0.25254
3,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.09070
4,1,2012-10-08,4296.95,Pharmacy,Grocery,32,16,7,0.21875,Digital Gold,...,900.83,5699.2,0.0,7029.92,4274.4,3117.06,0,18,Grocery,0.04403


In [8]:
import pandas as pd

# Assuming the data is stored in a pandas DataFrame called 'df'
# and the cashback percentage column is named 'cashback_percentage'

# Define the range intervals
intervals = [(0, 0.05), (0.05, 0.1), (0.1, 0.15), (0.15, 0.2), (0.2, 0.25), (0.25, 0.3), (0.3, 0.35), (0.35, 0.4), (0.4, 0.45), (0.45, 0.5)]

# Initialize a dictionary to store the counts for each range
counts = {}

# Iterate through the intervals
for interval in intervals:
    # Filter the rows that fall into the current interval
    filtered_rows = df[(df['cashback_percentage'] >= interval[0]) & (df['cashback_percentage'] < interval[1])]
    
    # Count the number of rows in the filtered DataFrame
    count = len(filtered_rows)
    
    # Store the count in the dictionary
    counts[interval] = count

# Print the counts for each range
for interval, count in counts.items():
    print(f"Total rows with cashback percentage between {interval[0]}-{interval[1]}: {count}")


Total rows with cashback percentage between 0-0.05: 9810
Total rows with cashback percentage between 0.05-0.1: 9853
Total rows with cashback percentage between 0.1-0.15: 9665
Total rows with cashback percentage between 0.15-0.2: 10267
Total rows with cashback percentage between 0.2-0.25: 10194
Total rows with cashback percentage between 0.25-0.3: 10064
Total rows with cashback percentage between 0.3-0.35: 364
Total rows with cashback percentage between 0.35-0.4: 343
Total rows with cashback percentage between 0.4-0.45: 326
Total rows with cashback percentage between 0.45-0.5: 373


In [9]:
import random

# Set random seed for reproducibility
random.seed(42)

# Update redemption_status for rows with cashback_percentage between 0 and 0.05
df.loc[(df['cashback_percentage'] >= 0) & (df['cashback_percentage'] < 0.05), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0) & (df['cashback_percentage'] < 0.05), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.01 else 0)

# Update redemption_status for rows with cashback_percentage between 0.05 and 0.1
df.loc[(df['cashback_percentage'] >= 0.05) & (df['cashback_percentage'] < 0.1), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.05) & (df['cashback_percentage'] < 0.1), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.025 else 0)

# Update redemption_status for rows with cashback_percentage between 0.1 and 0.15
df.loc[(df['cashback_percentage'] >= 0.1) & (df['cashback_percentage'] < 0.15), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.1) & (df['cashback_percentage'] < 0.15), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.03 else 0)

# Update redemption_status for rows with cashback_percentage between 0.15 and 0.2
df.loc[(df['cashback_percentage'] >= 0.15) & (df['cashback_percentage'] < 0.2), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.15) & (df['cashback_percentage'] < 0.2), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.045 else 0)

# Update redemption_status for rows with cashback_percentage between 0.2 and 0.25
df.loc[(df['cashback_percentage'] >= 0.2) & (df['cashback_percentage'] < 0.25), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.2) & (df['cashback_percentage'] < 0.25), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.055 else 0)

# Update redemption_status for rows with cashback_percentage between 0.25 and 0.3
df.loc[(df['cashback_percentage'] >= 0.25) & (df['cashback_percentage'] < 0.3), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.25) & (df['cashback_percentage'] < 0.3), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.07 else 0)

# Update redemption_status for rows with cashback_percentage between 0.3 and 0.35
df.loc[(df['cashback_percentage'] >= 0.3) & (df['cashback_percentage'] < 0.35), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.3) & (df['cashback_percentage'] < 0.35), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.09 else 0)

# Update redemption_status for rows with cashback_percentage between 0.35 and 0.4
df.loc[(df['cashback_percentage'] >= 0.35) & (df['cashback_percentage'] < 0.4), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.35) & (df['cashback_percentage'] < 0.4), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.105 else 0)

# Update redemption_status for rows with cashback_percentage between 0.4 and 0.45
df.loc[(df['cashback_percentage'] >= 0.4) & (df['cashback_percentage'] < 0.45), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.4) & (df['cashback_percentage'] < 0.45), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.125 else 0)

# Update redemption_status for rows with cashback_percentage between 0.45 and 0.5
df.loc[(df['cashback_percentage'] >= 0.45) & (df['cashback_percentage'] < 0.5), 'redemption_status'] = \
    df.loc[(df['cashback_percentage'] >= 0.45) & (df['cashback_percentage'] < 0.5), 'redemption_status'].apply(
        lambda x: 1 if random.random() <= 0.14 else 0)


In [10]:
df['redemption_status'].value_counts()

0    58699
1     2560
Name: redemption_status, dtype: int64

In [13]:
df.to_csv('new_data.csv')